In [1]:
import os , sys 
sys.path.append("./../")

In [2]:
from src.models.custom_http.llm import LLMAPI

In [3]:
llm = LLMAPI(
        llm_type="ollama",
        host_name="http://localhost:11434",
        endpoint="/api/generate",
        request_timeout=600,
        max_retries=3,
        params={
            "model": "deepseek-coder:6.7b",
            # "num_predict": 300,
        },
    )

In [4]:
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate


In [ ]:

def transform_func(inputs: dict) -> dict:
    import re

    text = inputs["text"]

    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r"(\r\n|\r|\n){2,}", r"\n", text)
    text = re.sub(r"[ \t]+", " ", text)

    return {"output_text": text}

clean_extra_spaces_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=transform_func
)
template = """Paraphrase this text:

{output_text}

In the style of a {style}.

Paraphrase: """
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain

prompt = PromptTemplate(input_variables=["style", "output_text"], template=template)
style_paraphrase_chain = LLMChain(llm=llm, prompt=prompt, output_key="final_output")

sequential_chain = SequentialChain(
    chains=[clean_extra_spaces_chain, style_paraphrase_chain],
    input_variables=["text", "style"],
    output_variables=["final_output"],
)
input_text = """
Chains allow us to combine multiple 


components together to create a single, coherent application. 

For example, we can create a chain that takes user input,       format it with a PromptTemplate, 

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by 


combining chains with other components.
"""
print("=" * 30)
print(sequential_chain.run({"text": input_text, "style": "a 90s rapper"}))
print("=" * 30)

In [5]:
llm_math = LLMMathChain(llm=llm, verbose=True)
print("=" * 30)
print(llm_math.run("What is 13 raised to the .3432 power?"))
print("=" * 30)
print(llm_math.prompt.template)

c:\Users\leesu\anaconda3\Lib\site-packages\langchain\chains\llm_math\base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?```text
13 ** .3432
```
...numexpr.evaluate("13 ** .3432")...

Answer: 2.4116004626599237
> Finished chain.
Answer: 2.4116004626599237
Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



In [5]:
from sqlalchemy import MetaData

metadata_obj = MetaData()
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)



from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [6]:

from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [7]:

for obs in observations:
    insert_obs(obs)

In [8]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

c:\Users\leesu\anaconda3\Lib\site-packages\langchain_experimental\sql\base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [9]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=3
)
     

In [10]:
query = (
    "What is the multiplication of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " +
    "between the same stock prices in January the 4th?"
)
agent_executor(query)

c:\Users\leesu\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQL Agent Executor chain...
To find the required ratio, we need to get stock prices of 'ABC' and 'XYZ' on January 3rd (let’s call it ABC_price) and January 4th (let’s call it XYZ_Price).

Next, we will compute their ratios. The formula for the ratio between two numbers a and b is:
Ratio = a / b

We can then subtract this ratio from 1 to find its reciprocal:
Reciprocal = 1 / Ratio

Action: sql_db_query
Action Input: "Select `stock_price` FROM `stock_prices` WHERE `ticker_symbol` = 'ABC' AND DATE(`date`) = '2023-01-03'"Error: (sqlite3.OperationalError) no such table: stock_prices
[SQL: Select `stock_price` FROM `stock_prices` WHERE `ticker_symbol` = 'ABC' AND DATE(`date`) = '2023-01-03']
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"],
    llm=llm
)

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

In [ ]:
query = (
    "What's the result of an investment of $10,000 growing at 8% annually for 5 years with compound interest?"
)
conversational_agent(query)

In [ ]:
# https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/06-langchain-agents.ipynb

In [ ]:
print(conversational_agent.agent.llm_chain.prompt.template)


In [ ]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer, Tool

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]
   

In [ ]:
docstore_agent = initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=3
)

In [ ]:
query = "What were Archimedes' last words?"
docstore_agent(query)